In [132]:
! pip install muspy
! pip install music21

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [133]:
import muspy as mp
import music21
import os

import random
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F

## Génération matrice A et B

In [134]:
def create_empty_notes_list(n : int):
  dict_note = []
  for i in range(0,n):
    dict_note.append(0.0)
  return dict_note

def normalize_list(l : list):
  total = sum(l)
  if total > 0.0:
    for i in range(len(l)):
      l[i] = l[i]/total

In [135]:
directory = "parts"
musics = []

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
      music_xml = mp.from_music21(music21.converter.parse(f))
      musics.append(music_xml)

/usr/local/lib/python3.8/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=1, channel=None, data=b'Viol\xe3o'>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=1, channel=None, data=b'1\xba Viol\xe3o'>; getting generic Instrument
  warnings.warn(


In [136]:
list_notes = []
list_notes_dur = []

for i in range(0,128): #on crée une liste pour chaque note qui servira à mettre la fréquence de ses notes voisines
  prob_allowed_notes = create_empty_notes_list(128)
  list_notes.append(prob_allowed_notes)

  prob_duration_notes = create_empty_notes_list(60)
  list_notes_dur.append(prob_duration_notes)

In [137]:
for music in musics:
  notes = music_xml.tracks[0].notes
  for i in range(0, len(notes)-1):
    list_to_update = list_notes[notes[i].pitch] #on récupère la liste des notes possibles de la note courante
    if notes[i].time != notes[i+1].time:
      list_to_update[notes[i+1].pitch] += 1

    list_to_update_dur = list_notes_dur[notes[i].pitch] #on récupère la liste des vélocités de la note courante
    list_to_update_dur[notes[i].duration] += 1

  list_to_update_dur = list_notes_dur[notes[len(notes)-1].pitch] 
  list_to_update_dur[notes[len(notes)-1].duration] += 1

In [138]:
for i in range(len(list_notes)):
  normalize_list(list_notes[i])
  normalize_list(list_notes_dur[i])

## Génération d'une séquence de notes par HMM


In [139]:
A = torch.tensor(list_notes)
A

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [140]:
B =torch.tensor(numpy.transpose(list_notes_dur))
B

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [141]:
L, V = torch.linalg.eig(A)
pi0 = V[:,0]
pi0 = pi0/torch.sum(pi0)
pi0 = torch.real(pi0)

In [183]:
N=600 # taille de la séquence que l'on veut
seq = [] #séquence de durées des notes
# initialisation
current_state = torch.multinomial(A.diagonal(),1).item()
obs = torch.multinomial(B[:,current_state],1).item()
seq.append(obs)

seq_states = [] #séquence des notes (pitch)
seq_states.append(current_state)

for i in range(N):
  # transition
  current_state = torch.multinomial(A[:,current_state],1).item()
  seq_states.append(current_state)
  
  #emission
  obs = torch.multinomial(B[:,current_state],1).item()
  seq.append(obs)

# optionnel
print(seq_states)
print(seq)

[76, 81, 72, 71, 66, 63, 71, 69, 74, 75, 72, 67, 69, 66, 62, 61, 62, 61, 62, 71, 76, 81, 72, 69, 74, 70, 62, 61, 62, 61, 62, 61, 62, 61, 62, 61, 62, 61, 62, 66, 63, 71, 69, 66, 63, 71, 68, 75, 72, 71, 68, 75, 70, 62, 61, 62, 61, 62, 61, 62, 61, 62, 66, 63, 71, 68, 71, 72, 64, 65, 74, 76, 74, 69, 66, 63, 71, 78, 73, 77, 79, 78, 73, 68, 75, 70, 65, 64, 79, 76, 73, 70, 62, 61, 62, 70, 65, 66, 62, 61, 62, 61, 62, 74, 70, 62, 61, 62, 61, 62, 61, 62, 70, 67, 64, 79, 78, 73, 77, 82, 77, 79, 78, 73, 70, 67, 64, 60, 64, 60, 64, 65, 62, 61, 62, 61, 62, 61, 62, 61, 62, 74, 77, 82, 77, 82, 77, 72, 69, 66, 63, 71, 68, 64, 67, 64, 60, 64, 60, 64, 60, 64, 60, 64, 65, 64, 60, 64, 65, 68, 72, 71, 66, 63, 71, 72, 69, 70, 67, 71, 66, 63, 71, 68, 75, 72, 69, 71, 68, 69, 70, 69, 66, 63, 71, 78, 73, 70, 65, 62, 61, 62, 61, 62, 66, 63, 71, 68, 71, 68, 75, 72, 69, 71, 68, 65, 62, 61, 62, 66, 63, 71, 70, 65, 64, 65, 68, 62, 61, 62, 61, 62, 61, 62, 61, 62, 61, 62, 61, 62, 70, 72, 69, 66, 63, 71, 78, 77, 79, 77,

## Génération de la partition

In [184]:
part = []
time = 0

for i in range(len(seq_states)):
  note = [time, seq_states[i], seq[i] + random.randint(0,10), 50 + random.randint(-10,10)]
  time += seq[i] + random.randint(0,10)
  part.append(note)

partition = numpy.array(part)
partition

array([[   0,   76,   30,   53],
       [  26,   81,   15,   58],
       [  39,   72,    9,   57],
       ...,
       [7152,   73,    7,   54],
       [7162,   68,   16,   41],
       [7170,   75,   14,   48]])

In [185]:
music_hmm = mp.to_music21(mp.from_note_representation(partition))

In [186]:
music_hmm.show('midi')

In [190]:
music_hmm.write("midi", fp="./content.midi")

'./content.midi'